# dbBOTPT

This notebook loads (or creates) a metadata database for the BOTPT data and plots some statistics.

## Starting from scratch
Example code showing how the database above was built. Since dbcamhd.json has been built, it is not normally necessary to run these cells. They can take a very long time without a large Dask worker cluster.

## Setup environment
Do these everytime running this notebook.

In [ ]:
%matplotlib inline
import xarray as xr
import numpy as np
import pandas as pd
import dask.dataframe as dd
import ooiod
import requests
import dask
from dask import delayed

### Start Dask cluster using Jupyterlab extension
Use the new Jupyterlab Dask extension buttons!

### Get a list of files from the server

In [ ]:
%%time
blob_urls = ooiod.blobs.list_blobs('ooiopendata', 'botpt')

In [ ]:
blob_urls[0:5]

### Get file sizes

In [ ]:
def get_blob_size(blob_url):
    try:
        header = requests.head(blob_url)
        blob_size = header.headers.get('Content-Length')
        return blob_size
    except:
        return None

In [ ]:
blob_sizes_delayed = []
for blob_url in blob_urls:
    blob_sizes_delayed.append(delayed(get_blob_size)(blob_url))

In [ ]:
%%time
blob_sizes = dask.compute(*blob_sizes_delayed)

### Create a Pandas dataframe from these Lists

In [ ]:
dbbotpt = pd.DataFrame(
    {'blob_url': blob_urls,
     'blob_size': blob_sizes,
    })
dbcamhd.tail()

In [ ]:
ds1 = xr.open_dataset(blob_urls[0])
ds1